# Caminhos e pacotes

In [1]:
import pandas as pd
pd.options.display.max_colwidth = 200

import os
import re
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
import soundfile as sf  
from pydub import AudioSegment
import math  
import shutil

###################################################################
#Para converter mp3 em wav
from pydub import AudioSegment
AudioSegment.converter = "C:\\ffmpeg\\bin\\ffmpeg.exe"
AudioSegment.ffmpeg = "C:\\ffmpeg\\bin\\ffmpeg.exe"
AudioSegment.ffprobe ="C:\\ffmpeg\\bin\\ffprobe.exe"
###################################################################

#Folder Inicial
path = os.getcwd()
pathin = path + '\\entrada\\'   
pathparcial = path + '\\parcial\\'

# Identificando arquivos

In [2]:
#Vendo todas as pastas e subpastas que tem
df = pd.DataFrame({'paths': [x[0] for x in os.walk(pathin)]})
df['count'] = df['paths'].apply(lambda x: len(re.findall(r"\\", x)))
print(df['count'].unique())
#Mantendo as pastas finais (mais profundas)
#df = df[df['count'] == max(df['count'].unique())]
df = df[df['count'] == 7]
df.reset_index(inplace = True)
print(df.shape)
df.head(3)

[6 7 8]
(14, 3)


,index,paths,count
0,2,C:\Users\laran\Downloads\BaixandoYoutube\entrada\Diogo\AlmaBrasileira,7
1,4,C:\Users\laran\Downloads\BaixandoYoutube\entrada\Diogo\Cuba,7
2,6,C:\Users\laran\Downloads\BaixandoYoutube\entrada\Diogo\SouEu,7


In [3]:
filepath = list()
filename = list()

for j in tqdm(df['paths']):
    mp3 = [f for f in os.listdir(j) if f.endswith('.mp3')]
    for k in mp3:
        filepath.extend([j + '\\'])
        filename.extend([k])

100%|████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 3506.94it/s]


In [4]:
data = pd.DataFrame({'path': filepath, 'mp3': filename})
data['wav'] = data['mp3'].apply(lambda x: x.replace('.mp3', '.wav'))

## Convertendo MP3 em WAV

In [5]:
#for i in tqdm(range(0, data.shape[0])):
#    src = data['path'][i] + data['mp3'][i]
#    dst = data['path'][i] + data['wav'][i]
    
#    # convert mp3 to wav (AudioSegment.from_mp3, AudioSegment.from_wav: também carregam o áudio, dando sound é só dar play)
#    sound = AudioSegment.from_mp3(src)
#    sound.export(dst, format = "wav")

## Criando subpasta para salvar arquivos menores

In [6]:
#Removendo as pastas, mesmo com arquivos (mesmo com read only files)
for i in data['path'].unique():
    shutil.rmtree(i + 'Menores\\', ignore_errors=True)

In [7]:
for i in data['path'].unique():
    try:
        os.stat(i + 'Menores\\')
    except:
        os.mkdir(i + 'Menores\\')

In [8]:
#Ouvindo 1 áudio de exemplo
#import IPython.display as ipd
#ipd.Audio(data['path'][0] + data['wav'][0]) # load a local WAV file

## Separando os áudios em áudios menores

In [9]:
#Quebrar em blocos de 30 segundos e excluir o último bloco (caso seja quebrado)
#Excluir também o primeiro que pode ser introdução

In [10]:
pathmenores = list()
normesoriginal = list()
nomesmenores = list()
comeco = list()
fim = list()

#Quebrar em 30 segundos
s = 30

for i in tqdm(range(0, data.shape[0])):
#for i in tqdm(range(0, 10)):
    
    Audio = AudioSegment.from_wav(data['path'][i] + data['wav'][i])
    
    #Divide por 1000 por se tratar de milisegundos
    #Quebrar em 30 segundos
    f = math.floor(len(Audio) / (1000 * s))
    
    for j in range(1,f): #Vai de 1 a (f - 1)
        t1 = j * s * 1000
        t2 = (j + 1) * s * 1000      
        
        newAudio = Audio[t1:t2]
        newAudio.export(data['path'][i] + 'Menores\\' + 'Musica_' + str(i) + 'Parte_' + str(j) + '.wav', format = "wav")
        
        pathmenores.extend([data['path'][i] + 'Menores'])
        normesoriginal.extend([data['wav'][i]])
        nomesmenores.extend(['Musica_' + str(i) + 'Parte_' + str(j) + '.wav'])
        comeco.extend([t1 / 1000])
        fim.extend([t2 / 1000])
        
df = pd.DataFrame({'path': pathmenores, 'fileoriginal': normesoriginal,  'file': nomesmenores,
                  'comeco': comeco, 'fim': fim})

joblib.dump(df, pathparcial + 'BaseDeDados.pkl')

df.shape

100%|████████████████████████████████████████████████████████████████████████████████| 199/199 [06:23<00:00,  3.77s/it]


(1161, 5)